In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.datasets import load_boston

boston = load_boston()
bostonDF = pd.DataFrame(boston.data, columns = boston.feature_names)
bostonDF['Price'] = boston.target
print(bostonDF.shape)
bostonDF.head(3)

In [ ]:
def get_update_weights_value(bias, w1,w2,rm, lstat,target,learning_rate=0.01):
    N = len(target)
    predict = w1 * rm + w2 * lstat + bias
    diff = target-predict
    bias_factors = np.ones((N,))
    
    w1_update = -(2/N)*learning_rate*(np.dot(rm.T, diff))
    w2_update = -(2/N)*learning_rate*(np.dot(lstat.T, diff))
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff))
    
    mse_loss = np.mean(np.square(diff))
    
    return bias_update, w1_update, w2_update, mse_loss

In [ ]:
def gradient_descent(features, target, iter_epochs=1000, verbose=True):
    
    #초기화
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.ones((1,))
    print(w1,w2,bias)
    learning_rate=0.01
    rm = features[:,0]
    lstat =features[:,1]
    
    for i in range(iter_epochs):
        bias_update, w1_update, w2_update, mse_loss = get_update_weights_value(bias, w1,w2,rm, lstat,target, learning_rate)
        w1 = w1-w1_update
        w2 = w2-w2_update
        bias = bias-bias_update
        if verbose:
            print('Epoch:', i+1,'/', iter_epochs)
            print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)
        
    return w1, w2, bias

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() #0에서 1사이 값으로 정규화
scaled_features = scaler.fit_transform(bostonDF[['RM','LSTAT']])

w1, w2, bias = gradient_descent(scaled_features,bostonDF['Price'],iter_epochs=1000,verbose=True)
print('###최종###')
print(w1,w2,bias)

In [ ]:
predicted = scaled_features[:,0]*w1 + scaled_features[:,1]*w2 + bias
bostonDF['PREDICTED_PRICE'] = predicted
bostonDF.head(5)

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential([
    Dense(1,input_shape=(2, ), activation=None, kernel_initializer='zeros',bias_initializer='ones')
])
model.compile(optimizer=Adam(learning_rate=0.01), loss='mse',metrics=['mse'])
model.fit(scaled_features, bostonDF['Price'].values,epochs=1000)

In [ ]:
bostonDF['KERAS_PREDICTED_PRICE'] = predicted
bostonDF.head(10)

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.datasets import load_boston

boston = load_boston()
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)
bostonDF['PRICE'] = boston.target
print(bostonDF.shape)
bostonDF.head()

In [ ]:
def get_update_weights_value_sgd(bias,w1,w2,rm_sgd,lstat_sgd,target_sgd,learning_rate=0.01):
    N = target_sgd.shape[0]
    predicted_sgd=w1*rm_sgd+w2*lstat_sgd+bias
    diff_sgd = target_sgd-predicted_sgd
    bias_factors = np.ones((N,))
    
    w1_update = -(2/N)*learning_rate*(np.dot(rm_sgd.T, diff_sgd))
    w2_update = -(2/N)*learning_rate*(np.dot(lstat_sgd.T, diff_sgd))
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff_sgd))
    
    return bias_update, w1_update, w2_update

In [ ]:
print(bostonDF['PRICE'].values.shape)
print(np.random.choice(bostonDF['PRICE'].values.shape[0], 1))
print(np.random.choice(506, 1))

In [ ]:
def st_gradient_descent(features, target, iter_epochs=1000, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정. 
    np.random.seed = 2021
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)
    
    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    
    
    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행. 
    for i in range(iter_epochs):
        # iteration 시마다 stochastic gradient descent 를 수행할 데이터를 한개만 추출. 추출할 데이터의 인덱스를 random.choice() 로 선택. 
        stochastic_index = np.random.choice(target.shape[0], 1)
        rm_sgd = rm[stochastic_index]
        lstat_sgd = lstat[stochastic_index]
        target_sgd = target[stochastic_index]
        # SGD 기반으로 Weight/Bias의 Update를 구함.  
        bias_update, w1_update, w2_update = get_update_weights_value_sgd(bias, w1, w2, rm_sgd, lstat_sgd, target_sgd, learning_rate)
        
        # SGD로 구한 weight/bias의 update 적용. 
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose:
            print('Epoch:', i+1,'/', iter_epochs)
            # Loss는 전체 학습 데이터 기반으로 구해야 함.
            predicted = w1 * rm + w2*lstat + bias
            diff = target - predicted
            mse_loss = np.mean(np.square(diff))
            print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)
        
    return w1, w2, bias

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']])

w1, w2, bias = st_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=5000, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(1,input_shape=(2,),activation=None, kernel_initializer='zeros',bias_initializer='ones')
])

model.compile(optimizer=Adam(learning_rate=0.01),loss='mse',metrics=['mse'])
model.fit(scaled_features,bostonDF['PRICE'].values, batch_size=30, epochs=1000)
predicted = model.predict(scaled_features)
bostonDF['KERAS_PREDICTED_PRICE_BATCH'] = predicted
bostonDF.head(10)